# Strands Agents を使用したエージェント群の作成

## マルチエージェントシステムと群知能の理解

エージェント群とは、自律的なAIエージェントが連携して複雑な問題を解決する集合体です。アリのコロニーや鳥の群れといった自然界のシステムに着想を得たエージェント群は、集合知能を活用し、その総合的な成果は単一のエージェントが生み出せるものを超えます。タスクを分散し、情報を共有することで、群は個々のエージェントが単独で動作するよりも効率的かつ効果的に複雑な問題に取り組むことができます。

マルチエージェントシステムは、環境内で相互作用する複数のインテリジェントエージェントで構成されます。これらのシステムにより、以下のことが可能になります。

- *分散型問題解決*: 複雑なタスクをサブタスクに分割し、並列処理する
- *情報共有*: エージェントが知見を共有し、集合知を構築する
- *特化*: 複数のエージェントが問題の特定の側面に焦点を当てる
- *冗長性*: 複数のエージェントが同様のタスクに取り組むことで、信頼性が向上する
- *創発的知能*: システムが個々のコンポーネントの能力を超えた能力を発揮する

群知能は以下のことに重点を置いています。

1. *分散制御*: 単一のエージェントがシステム全体を統制することはない
2. *ローカルインタラクション*: エージェントは主に近くのエージェントとインタラクトする
3. *単純なルール*: 個々のエージェントは比較的単純な動作に従う
4. *創発的複雑性*: 複雑なシステム動作は、単純なエージェント間のインタラクションから生じる

## 1. Swarm ツールを使ったクイックスタート

Strands Agents SDKには、マルチエージェントシステムの実装を簡素化し、ユーザーがすぐに使い始めることができる組み込みのSwarmツールが付属しています。このツールは共有メモリを実装します。

In [ ]:
!pip install -r requirements.txt

In [ ]:
from strands import Agent
from strands_tools import swarm
import time

### 1.1 ツールの直接呼び出し

In [ ]:
agent = Agent(tools=[swarm])

In [ ]:
result = agent.tool.swarm(
    task="この科学論文を分析し、重要な発見を特定してください。",
    swarm_size=5,
    coordination_pattern="collaborative",
)

ここでは、5人のエージェントが他のエージェントの洞察に基づいて合意形成を図っていることがわかります。

In [ ]:
result["content"][2:]

競争力が必要なシナリオはどうでしょうか?

In [ ]:
result = agent.tool.swarm(
    task="この科学論文を分析し、重要な発見を特定してください。",
    swarm_size=5,
    coordination_pattern="competitive",
)

In [ ]:
result["content"][2:]

群れは、専門化されたエージェントの役割を通じて、様々なコラボレーション戦略を実行します。

- 協調パターン：エージェントは他者の洞察を基に構築し、合意形成を目指します。
- 競争パターン：エージェントは独自の解決策と独自の視点を開発します。

### 1.2 自然言語呼び出し

In [ ]:
# 初期化
agent = Agent(tools=[swarm])

# 自然言語による呼び出し
result = str(agent(
    "4 つのエージェントの群れを使用して、生成 AI ベースのエージェントの現在の市場動向を分析します。"
))

In [ ]:
print(result)

Swarmツールは、Swarm内の全エージェントのための中央知識リポジトリとして機能するSharedMemoryシステムを実装しています。このシステムは、エージェントが自身の貢献をメタデータ（エージェントID、コンテンツ、フェーズ、タイムスタンプなど）とともに記録できるスレッドセーフなストアを維持します。処理フェーズを追跡することで、エージェントは現在のフェーズの知識のみを取得したり、履歴情報にアクセスしたりすることができます。この共有メモリアーキテクチャは、同時コラボレーションを可能にし、貢献履歴を維持し、エージェント間のスムーズな情報フローを確保します。これらはすべて、Swarmにおける効果的な集合知に不可欠な機能です。

Swarmツールの完全な実装は、[Strands Toolsリポジトリ](https://github.com/strands-agents/tools/blob/main/src/strands_tools/swarm.py) にあります。

主要なパラメータ
- task: スウォームが処理するメインタスク
- swarm_size: スウォーム内のエージェント数 (1～10)
- coordination_pattern: エージェントの連携方法
- collaboration: エージェントは他のエージェントの洞察に基づいて行動する
- competitive: エージェントは独立した解決策を開発する
- hybrid: 協力と独立した探索のバランスをとる

スウォームツールの動作
1. *初期化*: 共有メモリと専用エージェントを持つスウォームを作成する
2. *フェーズ処理*: エージェントはThreadPoolExecutorを使用して並列に動作する
3. *知識共有*: エージェントは共有メモリに情報を保存および取得する
4. *結果収集*: すべてのエージェントからの結果が集約され、表示される


## 2. Strandsエージェントで群れを作る

Strands Agents SDK を使用すると、既存の Agent オブジェクトを使用して、異なるモデルプロバイダーや異なる構成を持つ Swarm を作成できます。階層型、並列型、シーケンシャル型、メッシュ型など、さまざまな通信アーキテクチャが考えられますが、以下の例は、エージェント間通信のための柔軟な基盤を提供するメッシュアーキテクチャの実装を示しています。

### 2.1 Mesh Swarm アーキテクチャ

<p align="center">
    <img src="./images/swarm_example.png">
</p>

メッシュアーキテクチャでは、すべてのエージェントが互いに直接通信できます。次の例は、メッシュ通信を用いて協調的に問題を解決する、特化型エージェントの群れを示しています。

In [ ]:
# 異なる専門知識を持つ専門エージェントを作成する
research_agent = Agent(system_prompt=("""あなたは情報収集と分析を専門とするリサーチエージェントです。
スウォームにおけるあなたの役割は、トピックに関する事実に基づいた情報と調査に基づく洞察を提供することです。
正確なデータを提供し、問題の重要な側面を特定することに重点を置く必要があります。
他のエージェントから情報を得た際は、その情報があなたの調査内容と一致しているかどうかを評価してください。
"""), 
callback_handler=None)

creative_agent = Agent(system_prompt=("""あなたは革新的なソリューションを生み出すことに特化したクリエイティブエージェントです。
スウォームにおけるあなたの役割は、既成概念にとらわれず、クリエイティブなアプローチを提案することです。
他のエージェントから得た情報を活用しつつ、独自のクリエイティブな視点を加えることが重要です。
他のエージェントが考えつかなかったような斬新なアプローチに焦点を当ててください。
"""), 
callback_handler=None)

critical_agent = Agent(system_prompt=("""あなたは、提案を分析し、欠陥を見つけることに特化したクリティカルエージェントです。
スウォームにおけるあなたの役割は、他のエージェントが提案した解決策を評価し、潜在的な問題を特定することです。
提案された解決策を注意深く検討し、弱点や見落としを見つけ、改善を提案してください。
最終的な解決策が堅牢であることを保証しながら、建設的な批判を心がけてください。
"""), 
callback_handler=None)

summarizer_agent = Agent(system_prompt="""あなたは、情報の統合を専門とするサマライザーエージェントです。
スウォームにおけるあなたの役割は、すべてのエージェントから洞察を集め、最終的な解決策をまとめることです。
最良のアイデアを組み合わせ、批判にも対処することで、包括的な回答を作成する必要があります。
元のクエリに効果的に対応する、明確で実用的な要約を作成することに注力してください。
""")

メッシュ通信は、エージェント間のメッセージを追跡するための辞書を使用して実装されます。

In [ ]:
# エージェント間のメッセージを追跡するための辞書（メッシュ通信）
messages = {
    "research": [],
    "creative": [],
    "critical": [],
    "summarizer": []
}

群れは複数のフェーズで動作し、各エージェントは最初に個別に問題を分析します。

In [ ]:
query = "Generative Ai"

In [ ]:
# フェーズ1：各専門エージェントによる初期分析
research_result = research_agent(query)
time.sleep(30)
creative_result = creative_agent(query)
time.sleep(30)
critical_result = critical_agent(query)

初期分析の後、結果は他のすべてのエージェントと共有されます (メッシュ通信)

In [ ]:
# 他のすべてのエージェントと結果を共有する（メッシュ通信）
messages["creative"].append(f"From Research Agent: {research_result}")
messages["critical"].append(f"From Research Agent: {research_result}")
messages["summarizer"].append(f"From Research Agent: {research_result}")

messages["research"].append(f"From Creative Agent: {creative_result}")
messages["critical"].append(f"From Creative Agent: {creative_result}")
messages["summarizer"].append(f"From Creative Agent: {creative_result}")

messages["research"].append(f"From Critical Agent: {critical_result}")
messages["creative"].append(f"From Critical Agent: {critical_result}")
messages["summarizer"].append(f"From Critical Agent: {critical_result}")

第 2 フェーズでは、各エージェントは他のすべてのエージェントからの入力に基づいてソリューションを改良します。

In [ ]:
# フェーズ2: 各エージェントは他のエージェントからの入力に基づいて改良
research_prompt = f"{query}\n\nConsider these messages from other agents:\n" + "\n\n".join(messages["research"])
creative_prompt = f"{query}\n\nConsider these messages from other agents:\n" + "\n\n".join(messages["creative"])
critical_prompt = f"{query}\n\nConsider these messages from other agents:\n" + "\n\n".join(messages["critical"])

refined_research = research_agent(research_prompt)

time.sleep(30)

refined_creative = creative_agent(creative_prompt)

time.sleep(30)

refined_critical = critical_agent(critical_prompt)

# 絞り込んだ結果をサマライザーと共有
messages["summarizer"].append(f"From Research Agent (Phase 2): {refined_research}")
messages["summarizer"].append(f"From Creative Agent (Phase 2): {refined_creative}")
messages["summarizer"].append(f"From Critical Agent (Phase 2): {refined_critical}")

最後に、サマライザー エージェントはすべての入力を統合して包括的なソリューションを作成します。

In [ ]:
# 最終段階: Summarizer が最終ソリューションを作成します
time.sleep(30)

summarizer_prompt = f"""
元のクエリ: {query}

すべてのエージェントからの以下の入力を総合的に最終ソリューションに統合してください:

{"\n\n".join(messages["summarizer"])}

研究結果、独創的なアイデア、そして批判的なフィードバックを取り入れた、構成のしっかりした最終回答を作成します。
"""

final_solution = str(summarizer_agent(summarizer_prompt))

In [ ]:
print(final_solution)

このメッシュアーキテクチャにより、すべてのエージェント間の直接的な通信が可能になり、各エージェントは他のすべてのエージェントと洞察を共有できます。専門分野（リサーチ、クリエイティブ、クリティカル、サマライザー）が連携して、多様な視点と反復的な改善によるメリットを享受できる包括的なソリューションを生み出します。

### 2.2 共有メモリの実装

メッシュ通信の例はエージェント間の連携を効果的に示していますが、共有メモリシステムを導入することで、以下の機能を提供することで群集の能力をさらに強化できます。

- 全エージェントのための一元化された知識リポジトリ
- 自動フェーズ追跡と履歴知識の保存
- スレッドセーフな同時アクセスによる効率向上
- 複数のインタラクションにわたる洞察の永続的な保存

共有メモリを使用してメッシュ群集の例を拡張することで、メッセージディクショナリをSharedMemoryインスタンスに置き換えることができ、コードを簡素化すると同時に、より高度な知識管理が可能になります。

## Swarm を使用する場合:

- 単一の複雑なタスクを迅速に並列処理する場合
- 同じ問題に対して複数の視点が必要な場合
- 集合知の恩恵を受けるタスクの場合

## 結論

マルチエージェントの群は、集合知によって複雑な問題を解決します。Strands Agents SDK は、カスタム実装と共有メモリを備えた組み込みの Swarm ツールの両方をサポートしています。タスクを専門のエージェントに分散し、効果的な通信を可能にすることで、Swarm は単一のエージェントが単独で動作するよりも優れた結果を実現します。メッシュ通信パターンを使用する場合でも、Swarm ツールを使用する場合でも、開発者は複数のエージェントが定義された役割、調整メカニズム、そして知識共有に基づいて連携するシステムを構築できます。

